In [20]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [21]:
headers={'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win 64 ; x64) Apple WeKit /537.36(KHTML , like Gecko) Chrome/80.0.3987.162 Safari/537.36'}
webpage=requests.get("https://internshala.com/internships/",headers=headers).text

In [22]:
#Send request and parse the HTML content
soup = BeautifulSoup(webpage, "html.parser")

In [23]:
#To Find Individual Internship Container
First_Page_Int=soup.find_all('div',class_='individual_internship')

# **Step1 : First Page All Internship Data**

In [24]:
Job_Title = []
Company_Name = []
Job_Location=[]
Job_Duration=[]
Stipend=[]
Posted_Status=[]
Internship_URL = []

for i in First_Page_Int:

  Job_Title_tag = i.find('a',class_='job-title-href')
  Job_Title.append(Job_Title_tag.text.strip() if Job_Title_tag else '')

  # Internship Detail URL
  if Job_Title_tag and Job_Title_tag.has_attr('href'):
    full_url = "https://internshala.com" + Job_Title_tag['href'].strip()
    Internship_URL.append(full_url)
  else:
    Internship_URL.append('')

  Company_Name_tag = i.find('p' , class_='company-name')
  Company_Name.append(Company_Name_tag.text.strip() if Company_Name_tag else '')

  Job_Location_tag = i.find('div',class_='locations') # Changed find_all to find
  Job_Location.append(Job_Location_tag.text.strip() if Job_Location_tag else '')

  Job_Duration_tag = i.find('i', class_='ic-16-calendar')
  Job_Duration.append(Job_Duration_tag.find_next_sibling('span').text.strip() if Job_Duration_tag else '')

  Stipend_tag = i.find('span',class_='stipend') # Changed find_all to find
  Stipend.append(Stipend_tag.text.strip() if Stipend_tag else '')

  Posted_Status_tag = i.find('div',class_='status-success') # Changed find_all to find
  Posted_Status.append(Posted_Status_tag.text.strip() if Posted_Status_tag else '')

df_First=pd.DataFrame({'Job_Title':Job_Title,
   'Company_Name':Company_Name,
   'Job_Location':Job_Location,
   'Job_Duration':Job_Duration,
   'Stipend':Stipend,
   'Posted_Status':Posted_Status,
   'Internship_URL': Internship_URL
   })

In [25]:
df_First.shape

(51, 7)

In [26]:
df.sample(5)

,Job_Title,Company_Name,Job_Location,Job_Duration,Stipend,Posted_Status,Internship_URL
12,Business Development (Sales),CollegeDekho.com,Jaipur,4 Months,"₹ 15,000 - 25,000 /month",,https://internshala.com/internship/detail/busi...
37,Hotel Management(Chef),Licious,Bangalore,4 Months,"₹ 15,000 /month",,https://internshala.com/internship/detail/hote...
24,Content Development (English),Marpu Foundation,Work from home,1 Month,Unpaid,2 days ago,https://internshala.com/internship/detail/work...
31,Data Analytics,Futurionic Private Limited,"Delhi, Gurgaon, Noida",6 Months,"₹ 13,000 - 15,000 /month",2 days ago,https://internshala.com/internship/detail/data...
45,Human Resources (HR),Innominds Software Private Limited,Hyderabad,6 Months,"₹ 5,000 - 5,500 /month",,https://internshala.com/internship/detail/part...


In [27]:
#First Page Company Names
Company_Name

['Kaizen Infotech Solutions Private Limited',
 'Evolution Educare Private Limited',
 'DigiVibrant Techsolutions Private Limited',
 'ACMY & Associates',
 'Nurture Xcel',
 'Maxxcell Institute Of Professional Studies Private Limited',
 'Adds',
 'The Breadcrumbs Digital',
 'LeadNexus LLP',
 'Diasimos Private Limited',
 '',
 'BrightCHAMPS',
 'CollegeDekho.com',
 'Decathlon Sport India Private Limited',
 'Focus Wise Solution',
 'Hungama Digital Media Entertainment Private Limited',
 'Marpu Foundation',
 'Marpu Foundation',
 'Marpu Foundation',
 'Marpu Foundation',
 'Marpu Foundation',
 'Marpu Foundation',
 'Marpu Foundation',
 'Marpu Foundation',
 'Marpu Foundation',
 'Marpu Foundation',
 'Vismay',
 'MediBuddy',
 'Hughes & Hughes Chem Ltd.',
 'IREED Academy India Private Limited',
 'Futurionic Private Limited',
 'ShineProjects India',
 'Carborundum Universal Limited',
 'Focus Wise Solution',
 'Growel Feeds Private Limited',
 'ROSSARI BIOTECH Limited',
 'Licious',
 'Cultfit Healthcare Private

# **Step2 : All Pages All Internship Data**

In [7]:
base_url = "https://internshala.com/internships/page-{}"

unique_links = set()
all_internships = []

page = 1

while True:
    print(f"Scraping Page {page}")
    response = requests.get(base_url.format(page), headers=headers)
    if response.status_code != 200:
        print("End of pages or error!")
        break

    soup = BeautifulSoup(response.content, "html.parser")
    containers = soup.find_all("div", class_="individual_internship")

    if not containers:
        print("No more internships found.")
        break

    found_new_internship = False

    for container in containers:
        internship = {}
        title_tag = container.find('a', class_='job-title-href')
        if title_tag and title_tag.has_attr('href'):
            internship_url = "https://internshala.com" + title_tag['href'].strip()

            # Skip if already scraped
            if internship_url in unique_links:
                continue

            unique_links.add(internship_url)
            internship['url'] = internship_url
            internship['title'] = title_tag.text.strip() if title_tag else ''
        else:
            continue  # Skip entries without URL

        # Company
        company_tag = container.find('p', class_='company-name')
        internship['company'] = company_tag.text.strip() if company_tag else ''

        # Location
        location_tag = container.find('div', class_='locations')
        internship['location'] = location_tag.text.strip() if location_tag else ''

        # Duration
        duration_items = container.find_all('div', class_='row-1-item')
        internship['duration'] = duration_items[1].text.strip() if len(duration_items) > 1 else ''

        # Stipend
        stipend_tag = container.find('span', class_='stipend')
        if stipend_tag:
            stipend_clean = stipend_tag.text.strip().replace("â‚¹", "₹")
            internship['stipend'] = stipend_clean
        else:
            internship['stipend'] = ''

        # Posted time
        posted_tag = container.find('div', class_='status-success')
        internship['posted'] = posted_tag.text.strip() if posted_tag else ''

        all_internships.append(internship)
        found_new_internship = True

    if not found_new_internship:
        print("No new internships found on this page.")
        break

    page += 1
    time.sleep(2)
len(all_internships), list(all_internships[0].keys()) if all_internships else []


Scraping Page 1
Scraping Page 2
Scraping Page 3
Scraping Page 4
Scraping Page 5
Scraping Page 6
Scraping Page 7
Scraping Page 8
Scraping Page 9
Scraping Page 10
Scraping Page 11
Scraping Page 12
Scraping Page 13
Scraping Page 14
Scraping Page 15
Scraping Page 16
Scraping Page 17
Scraping Page 18
Scraping Page 19
Scraping Page 20
Scraping Page 21
Scraping Page 22
Scraping Page 23
Scraping Page 24
Scraping Page 25
Scraping Page 26
Scraping Page 27
Scraping Page 28
Scraping Page 29
Scraping Page 30
Scraping Page 31
Scraping Page 32
Scraping Page 33
Scraping Page 34
Scraping Page 35
Scraping Page 36
Scraping Page 37
Scraping Page 38
Scraping Page 39
Scraping Page 40
Scraping Page 41
Scraping Page 42
Scraping Page 43
Scraping Page 44
Scraping Page 45
Scraping Page 46
Scraping Page 47
Scraping Page 48
Scraping Page 49
Scraping Page 50
Scraping Page 51
Scraping Page 52
Scraping Page 53
Scraping Page 54
Scraping Page 55
Scraping Page 56
Scraping Page 57
Scraping Page 58
Scraping Page 59
Scrapi

(8128,
 ['url', 'title', 'company', 'location', 'duration', 'stipend', 'posted'])

In [9]:
#convert it to dataframe
df = pd.DataFrame(all_internships)

In [10]:
#Convert To CSV file
df.to_csv('/content/internshala_internships.csv', index=False, encoding='utf-8-sig')

In [11]:
#Download the CSV file
from google.colab import files
files.download('/content/internshala_internships.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>